# Brain Tumor Detection

### For Google Colab Users

In [ ]:
#@title ## Mount Your Google Drive & Map your Directory
#@markdown Please run this cell (`Ctrl+Enter` or `Shift+Enter`) and follow the steps printed bellow.
import os

from google.colab import drive
drive.mount('/content/gdrive')

# check Google Drive is mounted
if not os.path.isdir("/content/gdrive"):
  raise FileNotFoundError("Your Google Drive isn't mounted. Please run the above cell.")

assignment_dir = "/content/gdrive/MyDrive/workspace/BrainTumorDetection"  #@param{type:"string"}
assignment_dest = "/content/project"

# create symbolic link
!rm -f {assignment_dest}
!ln -s "{assignment_dir}" "{assignment_dest}"
print(f'Succesfully mapped (ln -s) "{assignment_dest}" -> "{assignment_dir}"')

# cd to linked dir
%cd -q {assignment_dest}
print(f'Succesfully changed directory (cd) to "{assignment_dest}"')

Mounted at /content/gdrive
Succesfully mapped (ln -s) "/content/project" -> "/content/gdrive/MyDrive/workspace/BrainTumorDetection"
Succesfully changed directory (cd) to "/content/project"


### Init

In [1]:
#@title ##Init

#@markdown Import required modules and observe your resources

# auto reload
%load_ext autoreload
%autoreload 2

# import
import torch
import torchvision
from PIL import Image
import matplotlib.pyplot as plt
import random
import os
import wandb
wandb.login()

from experiment import Experiment
from post_process import get_masks_from_heatmaps, present_masks, calc_iou
from data import DataGenerator, SegmentationGenerator
from train import get_best_models_dict

# Show resources
!nvidia-smi

wandb: Currently logged in as: antebi-itai (use `wandb login --relogin` to force relogin)


Wed Jul 21 15:49:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   37C    P0    59W / 300W |  19006MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Experiment

In [3]:
# params
config = {
"model_name": "vgg19_seg",
#"data_train_class": DataGenerator,         "data_train_path": os.path.join(".", "Data", "Training"),
"data_train_class": SegmentationGenerator, "data_train_path": os.path.join(".", "SegmentationData", "train"), 
#"data_test_class": DataGenerator,          "data_test_path":  os.path.join(".", "Data", "Testing"),
"data_test_class": SegmentationGenerator,  "data_test_path":  os.path.join(".", "SegmentationData", "test"),
"mri_type": "T1c", 
"epochs": 5,
"train_batch_size": 100,
"lr": 1e-4,
"heatmap_batch_size": 256,
"occlusion_size": (30,30),
"input_size": (256, 256), 
"heat_layers" : [['features', layer_num] for layer_num in [0, 2, 5, 7, 10, 12, 14, 16, 19, 21, 23, 25, 28, 30, 32, 34]] +
                [['classifier', -1]],
"heatmap_threshold" : 0.80,
"smallest_contour_len" : 30,
"shuffle_data": True,
"device": "cuda",
"gt_threshold": 0, 
"attributes_to_log": ["model_name", "occlusion_size", "heatmap_threshold"]
}

with wandb.init(project="BrainTumorDetection", config=config):
    ### show possible models
    print("Possible models:")
    for model, accuracy in get_best_models_dict().items():
        print("Model {model} {spaces}-> accuracy {acc}".format(spaces=" "*(25-len(model)), model=model, acc=accuracy))
    print()
    
    ### load best model
    experiment = Experiment(config)
    print("Model's accuracy: ", "{:.2f}".format(experiment.eval_model()), flush=True)
    
    ### train the model
    #experiment.train_model()
    #print("Model's accuracy: ", "{:.2f}".format(experiment.eval_model()), flush=True)
    for image_num in [0, 1]:
        title="#{image_num}".format(image_num=image_num)
        
        ### original image
        original_image, (gt_mask, gt_) = experiment.test_dataset[image_num * 2 + 1]
        wandb.log({"{title}/original_image".format(title=title): [wandb.Image(original_image)]})
        
        ### generate heatmaps
        heatmaps = experiment.generate_heatmap(original_image, title=title)
        
        ### calculate masks
        hot_masks, cold_masks = get_masks_from_heatmaps(heatmaps, 
                                                        thresh=config["heatmap_threshold"], 
                                                        smallest_contour_len=config["smallest_contour_len"])
        present_masks(original_image, gt_mask, hot_masks, cold_masks, title=title, gt_threshold=config["gt_threshold"])
        
        ### calculate IOU
        setattr(experiment, "IOU", calc_iou(gt_mask.cpu().numpy(), cold_masks["['classifier', -1]"]))
        
        ### log hyperparameters
        experiment.log_hyperparameter(title=title, additional_attributes=["IOU"])

wandb: wandb version 0.11.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Possible models:
Model vgg19_seg                 -> accuracy 0.9166666865348816
Model vgg19                     -> accuracy 1.0

loading vgg19_seg model...
loading best model state from: ./Models/vgg19_seg.pth
Model's accuracy:  0.90


/home/labs/waic/itaian/.conda/envs/itai-env/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Creating heatmap...


100%|██████████| 256/256 [02:22<00:00,  1.80it/s]


Creating heatmap...


100%|██████████| 256/256 [02:22<00:00,  1.80it/s]


Test/accuracy/no,0.93333
_runtime,314
_timestamp,1626872103
_step,14
Test/accuracy/yes,0.86667
Test/accuracy,0.9


Test/accuracy/no,▁
_runtime,▁▁▁▁▄▄▄▄▄▅█████
_timestamp,▁▁▁▁▄▄▄▄▄▅█████
_step,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
Test/accuracy/yes,▁
Test/accuracy,▁


### Debug